In [1]:
!pip uninstall tensorflow
!pip install tensorflow==2.1.0
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available() == True)

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 3.9MB 58.8MB/s 
     |████████████████████████████████| 450kB 49.0MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully un

In [2]:
!wget http://www.gutenberg.org/cache/epub/11339/pg11339.txt

--2020-03-13 04:57:56--  http://www.gutenberg.org/cache/epub/11339/pg11339.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243023 (237K) [text/plain]
Saving to: ‘pg11339.txt’

pg11339.txt         100%[===================>] 237.33K   400KB/s    in 0.6s    

2020-03-13 04:57:57 (400 KB/s) - ‘pg11339.txt’ saved [243023/243023]



In [0]:
filename = "pg11339.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()

start = text.find("THE FOX AND THE GRAPES\n\n\n")
end = text.find("ILLUSTRATIONS\n\n\n[")
text = text[start:end]

In [15]:
text

'THE FOX AND THE GRAPES\n\n\nA hungry Fox saw some fine bunches of Grapes hanging from a vine that\nwas trained along a high trellis, and did his best to reach them by\njumping as high as he could into the air. But it was all in vain, for\nthey were just out of reach: so he gave up trying, and walked away\nwith an air of dignity and unconcern, remarking, "I thought those\nGrapes were ripe, but I see now they are quite sour."\n\n\n\n\nTHE GOOSE THAT LAID THE GOLDEN EGGS\n\n\nA Man and his Wife had the good fortune to possess a Goose which laid\na Golden Egg every day. Lucky though they were, they soon began to\nthink they were not getting rich fast enough, and, imagining the bird\nmust be made of gold inside, they decided to kill it in order to\nsecure the whole store of precious metal at once. But when they cut it\nopen they found it was just like any other goose. Thus, they neither\ngot rich all at once, as they had hoped, nor enjoyed any longer the\ndaily addition to their wealth.\n\

In [0]:
import re

seq_length = 20

start_story = '| ' * seq_length

text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

In [17]:
text

' | | | | | | | | | | | | | | | | | | | | the fox and the grapes . a hungry fox saw some fine bunches of grapes hanging from a vine that was trained along a high trellis , and did his best to reach them by jumping as high as he could into the air . but it was all in vain , for they were just out of reach : so he gave up trying , and walked away with an air of dignity and unconcern , remarking , " i thought those grapes were ripe , but i see now they are quite sour . " | | | | | | | | | | | | | | | | | | | | the goose that laid the golden eggs . a man and his wife had the good fortune to possess a goose which laid a golden egg every day . lucky though they were , they soon began to think they were not getting rich fast enough , and , imagining the bird must be made of gold inside , they decided to kill it in order to secure the whole store of precious metal at once . but when they cut it open they found it was just like any other goose . thus , they neither got rich all at once , as the

In [18]:
len(text)

213716

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=False , filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

In [23]:
total_words

4169

In [27]:
len(token_list)

50436

In [0]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [49]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    

    y = to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

Number of sequences: 50416 



In [50]:
X.shape

(50416, 20)

In [51]:
y.shape

(50416, 4169)

In [0]:
n_units = 256
embedding_size = 100

text_in = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(total_words,embedding_size)(text_in)
x = tf.keras.layers.LSTM(n_units)(x)
x = tf.keras.layers.Dropout(0.2)(x)
text_out = tf.keras.layers.Dense(total_words,activation='softmax')(x)

model = tf.keras.Model(text_in , text_out)

model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.RMSprop(lr= 0.001))
epochs=100
batch_size=32

In [59]:
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_7 (Embedding)      (None, None, 100)         416900    
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 4169)              1071433   
Total params: 1,853,901
Trainable params: 1,853,901
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(X,y,epochs=epochs, batch_size=batch_size,shuffle=True)

Train on 50416 samples
Epoch 1/100
50416/50416 [==============================] - 14s 281us/sample - loss: 5.0992
Epoch 2/100
50416/50416 [==============================] - 16s 318us/sample - loss: 4.7458
Epoch 3/100
50416/50416 [==============================] - 20s 406us/sample - loss: 4.5869
Epoch 4/100
50416/50416 [==============================] - 17s 338us/sample - loss: 4.4840
Epoch 5/100
50416/50416 [==============================] - 16s 325us/sample - loss: 4.4130
Epoch 6/100
50416/50416 [==============================] - 19s 372us/sample - loss: 4.3678
Epoch 7/100
50416/50416 [==============================] - 18s 365us/sample - loss: 4.3526
Epoch 8/100
50416/50416 [==============================] - 16s 318us/sample - loss: 4.3739
Epoch 9/100
50416/50416 [==============================] - 16s 315us/sample - loss: 4.3726
Epoch 10/100
50416/50416 [==============================] - 17s 334us/sample - loss: 4.3470
Epoch 11/100
50416/50416 [==============================] - 17s 34

In [0]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '
            
            
    return output_text

In [67]:
seed_text = "the frog and the snake . "
gen_words = 500
temp = 0.1
token_type='word'
print (generate_text(seed_text, gen_words, model, seq_length, temp))

the frog and the snake . a man of a tree were in a am of a other - tree and the sea - tree and said , " not a , . you had a 
